In [1]:
## magics

%load_ext version_information 
%version_information numpy,xarray,netCDF4

/home/users/albert6a/anaconda2/lib/python2.7/site-packages/xarray/core/formatting.py:16: FutureWarning: The pandas.tslib module is deprecated and will be removed in a future version.
  from pandas.tslib import OutOfBoundsDatetime


Software versions
Python 2.7.13 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 5.1.0
OS Linux 2.6.32 5 amd64 x86_64 with debian 6.0.10
numpy 1.12.1
xarray 0.9.1
netCDF4 1.2.4
Mon Aug 21 17:57:38 2017 CEST

In [1]:
## path for mdules

import sys
sys.path.insert(0,"/home/albert/lib/python")

In [2]:
## imports

import numpy as np
import dask
import xarray as xr
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import pandas as pd 
import datetime
import glob as glob
import matplotlib.gridspec as gridspec
from mpl_toolkits.basemap import Basemap
import time
from dask.diagnostics import ProgressBar

%matplotlib inline

/home/users/albert6a/anaconda2/lib/python2.7/site-packages/xarray/core/formatting.py:16: FutureWarning: The pandas.tslib module is deprecated and will be removed in a future version.
  from pandas.tslib import OutOfBoundsDatetime


In [3]:
## datasets

#NATL60

dir="/home/users/albert6a/Data/NATL60/"
file="NATL60-CJM165_y2012m06d14.1d_gridT.nc"

tfile=dir+file

!ls $tfile


/home/users/albert6a/Data/NATL60/NATL60-CJM165_y2012m06d14.1d_gridT.nc


In [4]:
dsN = xr.open_dataset(tfile)

In [5]:
dsN

<xarray.Dataset>
Dimensions:               (axis_nbounds: 2, deptht: 300, time_counter: 1, x: 5422, y: 3454)
Coordinates:
    nav_lat               (y, x) float32 26.5648 26.5648 26.5648 26.5648 ...
    nav_lon               (y, x) float32 -81.4512 -81.4346 -81.4179 -81.4012 ...
  * deptht                (deptht) float32 0.480455 1.55879 2.79421 4.18731 ...
    time_centered         (time_counter) datetime64[ns] 2012-06-14T12:00:00
  * time_counter          (time_counter) datetime64[ns] 2012-06-14T12:00:00
Dimensions without coordinates: axis_nbounds, x, y
Data variables:
    deptht_bounds         (deptht, axis_nbounds) float32 0.0 1.00003 ...
    votemper              (time_counter, deptht, y, x) float64 nan nan nan ...
    time_centered_bounds  (time_counter, axis_nbounds) float64 1.718e+09 ...
    time_counter_bounds   (time_counter, axis_nbounds) float64 1.718e+09 ...
    vosaline              (time_counter, deptht, y, x) float64 nan nan nan ...
    sossheig              (time_coun

In [6]:
latN = dsN.nav_lat
lonN = dsN.nav_lon
depN = dsN.deptht
timN = dsN.time_counter

In [7]:
observation_lon=-63.784
observation_lat=37.495

In [ ]:
k=0

model_temperature = dsN.votemper[0,k]
model_salinity = dsN.vosaline[0,k]

model_temperature_stacked = model_temperature.stack(profile=('x', 'y'))
model_salinity_stacked = model_salinity.stack(profile=('x', 'y'))
lonN_stacked = lonN.stack(profile=('x', 'y'))
latN_stacked = latN.stack(profile=('x', 'y'))

distance_threshold = 0.25
square_distance_to_observation = (lonN_stacked - observation_lon)**2 + (latN_stacked-observation_lat)**2
is_close_to_observation = square_distance_to_observation < distance_threshold**2

model_temperature_near_observation = model_temperature_stacked.where(is_close_to_observation,drop=True)
model_salinity_near_observation = model_salinity_stacked.where(is_close_to_observation, drop=True)
lat_near_observation = latN_stacked.where(is_close_to_observation, drop=True)
lon_near_observation = lonN_stacked.where(is_close_to_observation, drop=True)


In [10]:
truc=model_temperature_near_observation[0,0]
truc.values

MemoryError: 

Traceback
---------
  File "/home/users/albert6a/anaconda2/lib/python2.7/site-packages/dask/async.py", line 264, in execute_task
    result = _execute_task(task, data)
  File "/home/users/albert6a/anaconda2/lib/python2.7/site-packages/dask/async.py", line 245, in _execute_task
    args2 = [_execute_task(a, cache) for a in args]
  File "/home/users/albert6a/anaconda2/lib/python2.7/site-packages/dask/async.py", line 245, in _execute_task
    args2 = [_execute_task(a, cache) for a in args]
  File "/home/users/albert6a/anaconda2/lib/python2.7/site-packages/dask/async.py", line 245, in _execute_task
    args2 = [_execute_task(a, cache) for a in args]
  File "/home/users/albert6a/anaconda2/lib/python2.7/site-packages/dask/async.py", line 245, in _execute_task
    args2 = [_execute_task(a, cache) for a in args]
  File "/home/users/albert6a/anaconda2/lib/python2.7/site-packages/dask/async.py", line 245, in _execute_task
    args2 = [_execute_task(a, cache) for a in args]
  File 